# Remote Agents

When agentic AI application grows, and so does the number of agents required, it is unlikely all of them can be assembled and served in a trivial manner - for instance, in a single container. As complexity grows, it becomes more and more important to be able to scale the agents independently of the main application. Additionally, it may be desirable to reuse agents over different teams, and even applications.

These considerations are of course not unique to AI applications, and have been addressed in the past by the microservices architecture.

This is where the concept of remote agents comes in. Remote agents are agents that are not part of the main application, but are instead running in a separate process, possibly on a separate machine/container. The main application communicates with these agents over a network, using either HTTP or gRPC protocols.

## Usage

Remote agents we very easy to use. You can host any existing agents (even basic `Askable`s) as remote agents by using any of the `AskableHost` classes. Then, you can use the `RemoteAskable` class to switchin any agent with it remote counterpart.

**NOTE** these examples also include response streaming, but is not mandatory.

## REST

This notebook demonstrates how to use remote agents with REST protocol. Please looks at other notebooks for gRPC protocol variant.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.workflow import Workflow, Conversation
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)

In [3]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.remote.remote").setLevel(logging.DEBUG)

In [ ]:
agent1 = Agent(id="agent1", llm=llm, description="Call this agent for general purpose questions", system_message = """You are an AI assistant
    Your task is to help the user with their questions.
    Always respond with the best answer you can generate.
    If you don't know the answer, respond with "I don't know".
    Always be polite and helpful.
    """)

In [ ]:
from vanilla_aiagents.remote.remote import RESTHost, RemoteAskable, RESTConnection

host = RESTHost(askables=[agent1], host="127.0.0.1", port=5001)


In [ ]:
host.start()
connection = RESTConnection(url="http://localhost:5001")
remote = RemoteAskable(id="agent1",connection=connection)
flow = Workflow(askable=remote, conversation=Conversation())

In [ ]:
async for step in flow.run_stream("Which is the capital of France?"):
    print(step)

In [ ]:
host.stop()